In [1]:
import numpy as np

from experiments.utils.SimpleComparison import SimpleComparison
from hdimvis.algorithms.spring_force_algos.chalmers96_algo.Chalmers96 import Chalmers96
from hdimvis.visualise_layouts_and_metrics.plot import show_layout,show_generation_metrics
from hdimvis.algorithms.stochastic_ntet_algo.SNeD import SNeD
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
from pathlib import Path
from definitions import PROJECT_ROOT

In [ ]:
datasets = ["rna N3k", "coil20", 'globe','fashion mnist', 'mnist']
results = {dataset : {metric: [] for metric in ["Stress", "k-Means F1", "kNN"]} for dataset in datasets}